Needed Libraries

- Note about installing necessary packages
- Look up proper way to note libraries needed

In [2]:
# Data Analysis Packages
import numpy as np
import pandas as pd
import matplotlib as mpl
import sklearn

# -- Utility Packages
import bs4

# ---- Folium for Map Generation
import folium

# ---- Packages used for API calls
import requests
import json

# ---- os for drawing out API keys from enviroment variables
import os

from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [12]:
# Retrieve Secret and ID from Enviroment Variables
%load_ext dotenv
%dotenv

CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')
VERSION = '20180323'

Pull list of Cities from Orange County Website

In [3]:
r = requests.get('https://media.ocgov.com/about/infooc/links/oc/occities.asp')
soup = bs4.BeautifulSoup(r.text)

oc_city_list = []
for cityTag in soup.select('div.edtdiv ul > li div a'):
    city_string = str(cityTag.string)
    # Strip out 'City of'
    city_string = city_string[8:]
    oc_city_list.append(city_string)
oc_city_list

['Aliso Viejo',
 'Anaheim',
 'Brea',
 'Buena Park',
 'Costa Mesa',
 'Cypress',
 'Dana Point',
 'Fountain Valley',
 'Fullerton',
 'Garden Grove',
 'Huntington Beach',
 'Irvine',
 'La Habra',
 'La Palma',
 'Laguna Beach',
 'Laguna Hills',
 'Laguna Niguel',
 'Laguna Woods',
 'Lake Forest',
 'Los Alamitos',
 'Mission Viejo',
 'Newport Beach',
 'Orange',
 'Placentia',
 'Rancho Santa Margarita',
 'San Clemente',
 'San Juan Capistrano',
 'Santa Ana',
 'Seal Beach',
 'Stanton',
 'Tustin',
 'Villa Park',
 'Westminster',
 'Yorba Linda']

Restaurant list will be built from calls to Foursquare's Places API

The Endpoint will be search in the venues group (https://developer.foursquare.com/docs/api-reference/venues/search/)

Test API Endpoint with Aliso Viejo

In [26]:
url = "https://api.foursquare.com/v2/venues/search"

# Category ID for the Food category on Foursquare
catID = "4d4b7105d754a06374d81259"

params = dict(
    client_id = CLIENT_ID,
    client_secret = CLIENT_SECRET,
    v = VERSION,
    near = 'Aliso Viejo, CA',
    limit = 50,
    categoryId = catID,
    radius = 4000
)

resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

In [25]:
venues = data['response']['venues']
venues[0]

{'id': '565d2265498ee06d2a765f5c',
 'name': "Raising Cane's Chicken Fingers",
 'contact': {},
 'location': {'address': '26801 Aliso Creek Rd',
  'lat': 33.57295113035663,
  'lng': -117.72392728155775,
  'labeledLatLngs': [{'label': 'display',
    'lat': 33.57295113035663,
    'lng': -117.72392728155775},
   {'label': 'entrance', 'lat': 33.573033, 'lng': -117.723768}],
  'postalCode': '92656',
  'cc': 'US',
  'city': 'Aliso Viejo',
  'state': 'CA',
  'country': 'United States',
  'formattedAddress': ['26801 Aliso Creek Rd',
   'Aliso Viejo, CA 92656',
   'United States']},
 'categories': [{'id': '4d4ae6fc7a7b7dea34424761',
   'name': 'Fried Chicken Joint',
   'pluralName': 'Fried Chicken Joints',
   'shortName': 'Fried Chicken',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/friedchicken_',
    'suffix': '.png'},
   'primary': True}],
 'verified': True,
 'stats': {'tipCount': 0,
  'usersCount': 0,
  'checkinsCount': 0,
  'visitsCount': 0},
 'beenHere': {'count': 0,
 